In [1]:
#meta:tag=hide
%env METAFLOW_PROFILE=dev-valay
%env METAFLOW_UI_URL=


env: METAFLOW_PROFILE=dev-valay
env: METAFLOW_UI_URL=


In [2]:
#meta:tag=hide
import os
os.makedirs("temp_files", exist_ok=True)


# `@huggingface_hub`

<!-- START doctoc -->
<!-- END doctoc -->


The `@huggingface_hub` decorator simplifies the process of downloading, caching, and managing models from the Hugging Face Hub. It provides seamless integration between Metaflow's datastore and Hugging Face Hub's model repository. This decorator is a syntactic sugar over the `@checkpoint` decorator to easily cache/load models from HuggingFace Hub. All models are stored with same way the `@checkpoint` decorator stores checkpoints i.e. objects are stored under the namespace and step name. The decorator injects a `huggingface_hub` object into the `current` singleton. This object has two main properties: 

1. Exposes a `loaded` property that returns the path to the models loaded via the `@huggingface_hub` decorator's `load` parameter.
2. Provides an wrapper over the [huggingface_hub](https://github.com/huggingface/huggingface_hub)'s [snapshot_download](https://huggingface.co/docs/huggingface_hub/main/en/package_reference/file_download#huggingface_hub.snapshot_download) function. The `current.huggingface_hub.snapshot_download` function returns a reference to the model stored in the datastore. This reference can be used in subsequent steps using the `@model` decorator. 


## Usage Patterns

### Loading Static Repos For A `@step`

Some Flows might require statically hard-coded models/datasets coming from huggingface. The `load` parameter of the `@huggingface_hub` decorator can be used to load these models/datasets. This style of loading models is very useful when models/datasets don't change often and can be hard-coded into the Flow. The models specified in the `load` parameter are downloaded from huggingface hub and stored in the datastore if they are not already present. The path to the model/dataset loaded is accessible via the `loaded` property of the `huggingface_hub` object. The below example shows how to load a static model from huggingface hub and access it in a `@step`. 

The `load` parameter can take multiple form of arguments. Where the simplest for is a list of strings representing the `repo_id`. It can also take a list of dictionaries that provide all the arguments to the [snapshot_download](https://huggingface.co/docs/huggingface_hub/main/en/package_reference/file_download#huggingface_hub.snapshot_download) function. 

In [3]:
%%writefile temp_files/hub_deco_flow.py
#meta:tag=hide_output
from metaflow import FlowSpec, step, huggingface_hub, pypi_base, current
import os

@pypi_base(packages={"huggingface-hub": "0.16.4"})
class SimpleHFFlow(FlowSpec):
    
    @huggingface_hub(load=["bert-base-uncased"])
    @step
    def start(self):
        import os
        # Access the loaded model through current.huggingface_hub.loaded
        model_path = current.huggingface_hub.loaded["bert-base-uncased"]
        print(f"Model loaded at: {model_path}")
        print(f"Contents: {os.listdir(model_path)}")
        self.next(self.load_to_path)
    
    @huggingface_hub(load=[("bert-base-uncased", "./model_directory")])
    @step
    def load_to_path(self):
        import os
        # Access the loaded model through current.huggingface_hub.loaded
        model_path = current.huggingface_hub.loaded["bert-base-uncased"]
        print(f"Model loaded at: {model_path}")
        print(f"Contents: {os.listdir(model_path)}")
        self.next(self.end)
    

    @huggingface_hub(load=[
        {
            "repo_id": "bert-base-uncased",
            "allow_patterns": ["*.json", "tokenizer.txt"],
            "repo_type": "model"
        },
    ])
    @step
    def end(self):
        # Access the loaded model through current.huggingface_hub.loaded
        model_path = current.huggingface_hub.loaded["bert-base-uncased"]
        print(f"Model loaded at: {model_path}")
        print(f"Contents: {os.listdir(model_path)}")
        

if __name__ == "__main__":
    SimpleHFFlow()

Overwriting temp_files/hub_deco_flow.py


In [4]:
#meta:tag=hide_input
#meta:show_steps=start,load_to_path,end
! python temp_files/hub_deco_flow.py --environment=pypi run

Metaflow 2.12.36.post9-git09d02cb-dirty+obcheckpoint(0.1.4);ob(v1) executing SimpleHFFlow for user:valay@outerbounds.co


Validating your flow...
    The graph looks good!
Running pylint...


    Pylint is happy!
2024-12-11 07:13:34.971 Bootstrapping virtual environment(s) ...


2024-12-11 07:13:35.093 Virtual environment(s) bootstrapped!


2024-12-11 07:13:37.881 Workflow starting (run-id 7472):


...

2024-12-11 07:14:35.577 [7472/end/47524 (pid 2223804)] Task is starting.


2024-12-11 07:14:37.338 [7472/end/47524 (pid 2223804)] [@huggingface_hub] Loading model from datastore to /tmp/metaflow_hf_9b5c6e8800_yfsk2bdb. Model being loaded: mf.huggingface_hub/checkpoints/artifacts/SimpleHFFlow/end/26ec4b03ee0e/24105546e482/73eef1dc.0.9b5c6e8800.0


2024-12-11 07:14:38.566 [7472/end/47524 (pid 2223804)] Model loaded at: /tmp/metaflow_hf_9b5c6e8800_yfsk2bdb
2024-12-11 07:14:38.566 [7472/end/47524 (pid 2223804)] Contents: ['config.json', 'tokenizer.json', 'tokenizer_config.json']


2024-12-11 07:14:39.334 [7472/end/47524 (pid 2223804)] Task finished successfully.


2024-12-11 07:14:39.457 Done!


### Loading HF Repos Dynamically

In many cases, Huggingface models or datasets might be passed down as parameters to the Flow. This approach can make it challenging to load them using the `load` parameter of the `@huggingface_hub` decorator. For this case the `current.huggingface_hub` provides a `snapshot_download` function that can be used to download the model/dataset from huggingface hub and return a reference to the model/dataset. This reference can be used in subsequent steps using the `@model` decorator. The core difference between the `load` parameter and the `snapshot_download` function is that the `load` parameter is used to load static models/datasets while the `snapshot_download` function will return a reference that can be loaded in future steps. If the `force_download` parameter is passed to the `snapshot_download` function, it will bust the cache, download the model/dataset again and store it in the datastore. 

The below example shows how to load a dynamic model from huggingface hub and access it in a `@step`. 

In [5]:
%%writefile temp_files/hub_deco_flow_2.py
#meta:tag=hide_output
from metaflow import FlowSpec, step, current, huggingface_hub, model
import os

class SimpleHFFlow(FlowSpec):
    
    @huggingface_hub
    @step
    def start(self):
        # Download a small model from HuggingFace Hub
        self.hf_model_reference = current.huggingface_hub.snapshot_download(
            repo_id="bert-base-uncased",
            allow_patterns=["*.json"]  # Only download the config file to keep it light
        )
        print(f"Model Reference saved with key : %s" % self.hf_model_reference["key"])
        self.next(self.end)
    
    @model(load="hf_model_reference")
    @step
    def end(self):
        print(f"Model loaded at: {current.model.loaded['hf_model_reference']}")
        print(f"Contents: {os.listdir(current.model.loaded['hf_model_reference'])}")

if __name__ == "__main__":
    SimpleHFFlow()

Overwriting temp_files/hub_deco_flow_2.py


In [6]:
#meta:tag=hide_input
#meta:show_steps=start,end
! python temp_files/hub_deco_flow_2.py run

Metaflow 2.12.36.post9-git09d02cb-dirty+obcheckpoint(0.1.4);ob(v1) executing SimpleHFFlow for user:valay@outerbounds.co


Validating your flow...
    The graph looks good!
Running pylint...


    Pylint is happy!


2024-12-11 07:14:45.090 Workflow starting (run-id 7473):


2024-12-11 07:14:46.220 [7473/start/47526 (pid 2223882)] Task is starting.


...

2024-12-11 07:14:50.743 [7473/end/47527 (pid 2223935)] [@model] Loading Artifact with name `hf_model_reference` [type:checkpoint] with key: mf.huggingface_hub/checkpoints/artifacts/SimpleHFFlow/start/26ec4b03ee0e/a92ae9615600/8bcbe12d.0.9b5c6e8800.0


2024-12-11 07:15:16.468 [7473/end/47527 (pid 2223935)] [@model] Loaded artifact `hf_model_reference[type:checkpoint]` in 25.73 seconds


2024-12-11 07:15:16.491 [7473/end/47527 (pid 2223935)] Model loaded at: /tmp/metaflow_models_hf_model_reference_4yejtsvy
2024-12-11 07:15:16.491 [7473/end/47527 (pid 2223935)] Contents: ['README.md', 'config.json', '.gitattributes', 'LICENSE', 'flax_model.msgpack', 'tf_model.h5', 'tokenizer.json', 'pytorch_model.bin', 'vocab.txt', 'model.onnx', 'rust_model.ot', 'model.safetensors', 'tokenizer_config.json']


2024-12-11 07:15:20.901 [7473/end/47527 (pid 2223935)] Task finished successfully.


2024-12-11 07:15:21.016 Done!
